In [5]:
from surprise import SVD
from surprise import Dataset
from surprise import evaluate

# Load the movielens-100k dataset (download it if needed),
# and split it into 3 folds for cross-validation.
data = Dataset.load_builtin('ml-100k')
data.split(n_folds=3)

# We'll use the famous SVD algorithm.
algo = SVD()

# Evaluate performances of our algorithm on the dataset.
perf = evaluate(algo, data, measures=['RMSE'])

perf.items
print(perf)

Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.9424
------------
Fold 2
RMSE: 0.9418
------------
Fold 3
RMSE: 0.9431
------------
------------
Mean RMSE: 0.9424
------------
------------
        Fold 1  Fold 2  Fold 3  Mean    
RMSE    0.9424  0.9418  0.9431  0.9424  


In [9]:
# Retrieve the trainset.
trainset = data.build_full_trainset()

# Build an algorithm, and train it.
algo = SVD()
algo.train(trainset)

In [10]:
uid = str(196)  # raw user id (as in the ratings file). They are **strings**!
iid = str(302)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r=4, verbose=True)

user: 196        item: 302        r = 4.00   est = 4.14   {'was_impossible': False}


In [71]:
import pandas as pd
# Retrieve the trainset and the testset.
train_reader = Reader(line_format='item rating user', sep=',',skip_lines=1) 
trainset = Dataset.load_from_file('../data/data_train_surprise.csv',reader=test_reader)
trainset = trainset.build_full_trainset()
test_reader = Reader(line_format='item rating user', sep=',',skip_lines=1) 
testset = Dataset.load_from_file('../data/sampleSubmission_surprise.csv',reader=test_reader)

In [72]:
testset = testset.build_full_trainset()
#x = [x for (uid,iid,r) in test] 
#testset.split(n_folds=3)
# Build an algorithm, and train it.
algo = SVD()
algo.train(trainset)
predictions_svd = algo.test(testset)
rmse(predictions_svd)
dump('./dump_SVD', predictions_svd, trainset, algo_svd)
dump_obj_svd = pickle.load(open('./dump_SVD', 'rb'))

# rui: true, est: estimated.
df_svd = pd.DataFrame(dump_obj_svd['predictions'], columns=['uid', 'iid', 'rui', 'est', 'details'])    
df_svd['err'] = abs(df_svd.est - df_svd.rui)
print(df_svd)

TypeError: 'Trainset' object is not iterable

In [28]:
# Save submission
df_svd["Id"] = "r" + df_svd["iid"].map(str) + "_c" +df_svd["uid"].map(str)
df_svd["Prediction"] = df_svd["est"].clip(0,5).round()
df_svd_new = df_svd.drop(["iid","uid","est","details","rui"],1)
output_path = '../results/submission_surprise.csv'
df_svd_new.to_csv(output_path,columns=["Id","Prediction"],index=False)

In [56]:
from helpers import load_data
import scipy.sparse as sp
#ratings = load_data('../data/sampleSubmission.csv')
ratings = load_data('../data/data_train.csv')
rows, cols, ratings = sp.find(ratings)
rows = rows + 1
cols = cols + 1
test_pd = pd.DataFrame({'item':rows,'user':cols,'rating':ratings})
output_path = '../data/data_train_surprise.csv'
test_pd.to_csv(output_path,index=False)

number of items: 10000, number of users: 1000
